# serve

> Fill in a module description here

In [1]:
#| default_exp serve

In [2]:
#| hide
from nbdev.showdoc import *

In [20]:
#| export

import tempfile
import os
import io
import base64
import PIL
import random
from pprint import pprint

import torch
from diffusers import StableDiffusionPipeline

import pydantic


from datascience_toolkits.fastapi import create_app, serve

In [33]:
#| export

device = torch.device('cuda')

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16, revision="fp16")

def do_nothing(clip_input, images):
    return images, [0] * len(images)


pipe.safety_checker = do_nothing
pipe = pipe.to(device)

_ = pipe("", num_inference_steps=1)


In [ ]:
#| export 

class Request(pydantic.BaseModel):
    prompt: str
    steps: int = 1
    guidance_scale: float = 7.5
    negative_prompt: str = None
    batch_size: int = 1
    seed: int = None
    mode: str = "txt2img"
    init_image: str = None
    strength: float = 0.8
        


async def txt2img(req: Request):
    pprint(req)
    tempdir = tempfile.gettempdir()
    local_dir = os.path.join(tempdir, "test.png")
    
    seed = req.seed if req.seed else random.randint(1, 1000000000)
    torch.manual_seed(seed)

    pipe_out = pipe(
        prompt=req.prompt, 
        num_inference_steps=req.steps,
        guidance_scale=req.guidance_scale,
        negative_prompt=req.negative_prompt,
    )
    images = pipe_out.images

    
    b64images = []
    for img in images:
        buffered = io.BytesIO()
        img.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue())
        b64images.append(img_str.decode())
    return {
        'images': b64images,
        'seed': seed,
    }
    

route_info = [{
    "path": "/",
    "method": "post",
    "endpoint": txt2img
}]

app = create_app(route_info)
serve(app)

t=2022-10-24T03:08:40+0000 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040


http://e2d6-54-249-55-5.ngrok.io


INFO:     Started server process [28909]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


In [1]:
#| hide
import nbdev; nbdev.nbdev_export()